# Лабораторна робота №9. Стиснення даних. Жадібна стратегія на прикладі кодування Гафмена

Виконав - Огоновський Олександр

**Мета**:  навчитись реалізовувати алгоритм побудови дерева оптимальних кодів Гафмена на основі черги з пріоритетом (за допомогою купи) засобами Python.


## Побудувати дерево кодів Гафмена згідно з варіантом, виданим викладачем вручну і візуалізувати дерево.

З практичної роботи №9:

<img src="GRAF.png">


Закодуємо кожен символ на код:

- A -> 00
- B -> 01
- C -> 1

Для всього тексту отримуємо бітову послідовність довжини 55 біт ($10 \cdot 2 + 11 \cdot 2 + 13 \cdot 1 = 55$).


## Написати процедуру на Python, яка для вхідного повідомлення обчислює список символів chars та список їх частот freq

In [1]:

dictt = {}
inputt = "AAAAAAAAABBBBBBBBBBBCCCCCCCCCCCCC"
for i in inputt:
    if not i in dictt:
        dictt[i] = 1
    else:
        dictt[i] += 1
print(dictt)

chars = list(dictt.keys())
freq = list(dictt.values())

print(chars)
print(freq)


{'A': 9, 'B': 11, 'C': 13}
['A', 'B', 'C']
[9, 11, 13]


## Перевірити результат за допомогою коду, наведеному у методичці.

In [2]:
# A Вузол дерева Гафмана 
import heapq 


class node: 
    def __init__(self, freq, symbol, left=None, right=None): 
        # частота символу 
        self.freq = freq 

        # назва символу (символ) 
        self.symbol = symbol 

        # вузол ліворуч від поточного вузла 
        self.left = left 

        # вузол праворуч від поточного вузла 
        self.right = right 

        # напрямок дерева (0/1) 
        self.huff = '' 

    def __lt__(self, nxt): 
        return self.freq < nxt.freq 


# утиліта для виведення кодів Гафмена для  
#  всіх символів у щойно створеному дереві Гафмена 
def printNodes(node, val=''): 

    # Код Гафмена для поточного вузла  
    newVal = val + str(node.huff) 

# якщо вершина не є реберною вершиною 
# то пройти всередині неї 
    if(node.left): 
        printNodes(node.left, newVal) 
    if(node.right): 
        printNodes(node.right, newVal) 

        # якщо node є реберною вершиною тоді 
        # вивести його хаффманівський код 
    if(not node.left and not node.right): 
        print(f"{node.symbol} -> {newVal}") 

# список, що містить невикористані вершини 
nodes = [] 

# перетворення символів та частот 
# у вузли дерева Гафмена 

for x in range(len(chars)): 
    heapq.heappush(nodes, node(freq[x], chars[x])) 

while len(nodes) > 1: 

    # відсортувати всі вершини за зростанням 
    # на основі їх частоти 
    left = heapq.heappop(nodes) 
    right = heapq.heappop(nodes) 

    # присвоїти значення напрямку цим вузлам 
    left.huff = 0
    right.huff = 1

    # об'єднати 2 найменші вершини, щоб створити 
    # новий вузол як їхній батько 
    newNode = node(left.freq+right.freq, left.symbol+right.symbol, left, right) 

    heapq.heappush(nodes, newNode) 

# Друк кодів Гафмена
printNodes(nodes[0])

C -> 0
A -> 10
B -> 11


## Опрацювати самостійно тему [декодування Гафмена](https://www.geeksforgeeks.org/huffman-decoding/?ref=gcse) [40] і скориставшись наведеним там кодом, декодувати повідомлення, задане варіантом.


In [7]:
import heapq
from collections import defaultdict

# to map each character its huffman value
codes = {}

# To store the frequency of character of the input data
freq = defaultdict(int)

# A Huffman tree node
class MinHeapNode:
    def __init__(self, data, freq):
        self.left = None
        self.right = None
        self.data = data
        self.freq = freq

    def __lt__(self, other):
        return self.freq < other.freq

# utility function to print characters along with
# there huffman value
def printCodes(root, str):
    if root is None:
        return
    if root.data != '$':
        print(root.data, ":", str)
    printCodes(root.left, str + "0")
    printCodes(root.right, str + "1")

# utility function to store characters along with
# there huffman value in a hash table
def storeCodes(root, str):
    if root is None:
        return
    if root.data != '$':
        codes[root.data] = str
    storeCodes(root.left, str + "0")
    storeCodes(root.right, str + "1")

# function to build the Huffman tree and store it
# in minHeap
def HuffmanCodes(size):
    global minHeap
    for key in freq:
        minHeap.append(MinHeapNode(key, freq[key]))
    heapq.heapify(minHeap)
    while len(minHeap) != 1:
        left = heapq.heappop(minHeap)
        right = heapq.heappop(minHeap)
        top = MinHeapNode('$', left.freq + right.freq)
        top.left = left
        top.right = right
        heapq.heappush(minHeap, top)
    storeCodes(minHeap[0], "")

# utility function to store map each character with its
# frequency in input string
def calcFreq(str, n):
    for i in range(n):
        freq[str[i]] += 1

# function iterates through the encoded string s
# if s[i]=='1' then move to node->right
# if s[i]=='0' then move to node->left
# if leaf node append the node->data to our output string
def decode_file(root, s):
    ans = ""
    curr = root
    n = len(s)
    for i in range(n):
        if s[i] == '0':
            curr = curr.left
        else:
            curr = curr.right

        # reached leaf node
        if curr.left is None and curr.right is None:
            ans += curr.data
            curr = root
    return ans + '\0'

# Driver code
if __name__ == "__main__":
    minHeap = []
    str = inputt
    encodedString, decodedString = "", ""
    calcFreq(str, len(str))
    HuffmanCodes(len(str))
    print("Character With there Frequencies:")
    for key in sorted(codes):
        print(key, codes[key])

    for i in str:
        encodedString += codes[i]

    print("\nEncoded Huffman data:")
    print(encodedString)
    # Function call
    decodedString = decode_file(minHeap[0], encodedString)
    print("\nDecoded Huffman Data:")
    print(decodedString)

Character With there Frequencies:
A 10
B 11
C 0

Encoded Huffman data:
10101010101010101011111111111111111111110000000000000

Decoded Huffman Data:
AAAAAAAAABBBBBBBBBBBCCCCCCCCCCCCC 


# Контрольні питання

1. **Що таке жадібні алгоритми?**

    Жадібні алгоритми - це алгоритми, які на кожному кроці роблять локально оптимальний вибір, сподіваючись отримати глобально оптимальне розв’язання. Вони не переглядають усіх можливих варіантів, а обирають «найкращий» крок у даний момент.

2. **Що таке префіксний код? Який код використовується у коді Гафмена?**

    Префіксний код - це код, у якому жодне кодове слово не є префіксом іншого. Це гарантує однозначне декодування без роздільників.

    У алгоритмі Гафмена використовується оптимальний префіксний код змінної довжини.

3. **Як пов’язана структура даних «купа» зі структурою даних «черга з пріоритетами»?**

    Купа (heap) є ефективною реалізацією пріоритетної черги. У мін-купі найменший елемент знаходиться у корені. Пріоритетна черга працює через операції вставки та вилучення елемента з найвищим пріоритетом, які у купі виконуються за O(log n).

4. **Що таке стиснення даних і для чого воно використовується? Які його основні переваги?**

    Стиснення даних - це зменшення обсягу даних шляхом оптимального кодування.
    Використовується для:
        - економії місця на диску;
        - прискорення передавання даних;
        - зменшення навантаження на мережу й ресурси.
    Переваги: менший розмір файлів, швидше передавання, ефективніше використання ресурсів.

5. **Які кроки необхідно виконати для стиснення даних за допомогою алгоритму кодування Гафмена?**

    1) Підрахувати частоти символів.
    2) Створити пріоритетну чергу (мін-купу).
    3) Побудувати дерево Гафмена, об’єднуючи два вузли з мінімальними частотами.
    4) Призначити бітові коди: ліве ребро - 0, праве - 1.
    5) Замінити символи у тексті відповідними кодами.
    6) Зберегти дерево або частоти для декодування.

6. **Основні обмеження та недоліки алгоритму Гафмена. Чи можливо покращити його продуктивність?**

    Недоліки:
        - не оптимальний, коли частоти символів близькі;
        - потрібна попередня статистика;
        - вимагає зберігання дерева;
        - не враховує контекст.
    Покращення:
        - адаптивний Гафмен;
        - поєднання з LZ (DEFLATE);
        - використання арифметичного кодування.

7. **Альтернативні методи стиснення даних, що можуть конкурувати з Гафменом:**

    - арифметичне кодування;
    - кодування Фано–Шеннона;
    - LZ77, LZ78, LZW;
    - DEFLATE, Brotli, Zstd;
    - BWT + MTF.
8. **Практичні застосування алгоритмів стиснення даних:**

    - архіватори (ZIP, GZIP);
    - мережеві протоколи (HTTP/2 HPACK);
    - формати файлів (PNG, JPEG, PDF);
    - IoT та телеметрія;
    - бази даних та системи логування;
    - стиснення тексту й мультимедіа.
